In [13]:
import helper.visualization as viz
import io
import sys
import pandas as pd
import matplotlib.pyplot as plt
import tempfile
import numpy as np

In [15]:
data = []
preprocessed_data = None
transformed_data = None
# with tempfile.TemporaryFile('w') as summary_file:
for subdir, dirs, files in os.walk('.\data'):
    for file in files:
        curr_file_name = os.path.join(subdir, file)
        with io.open(curr_file_name) as curr_file:
            data.extend(curr_file.readlines())
preprocessed_data = (line.replace("\n", "") for line in data)
transformed_data = (line.split(",") for line in preprocessed_data)

aggregated_data = pd.DataFrame(transformed_data, columns=("Method", "N","µ","NMI"), dtype=np.float64)

,Method,N,µ,NMI
0,Coverage Maximization,250.0,0.1,1.000000
1,Coverage Maximization,250.0,0.2,0.525391
2,Coverage Maximization,250.0,0.3,0.429109
3,Coverage Maximization,250.0,0.4,0.000000
4,Coverage Maximization,250.0,0.5,0.000000
...,...,...,...,...
852,Random Propagation Algorithm,500.0,0.1,0.181878
853,Random Propagation Algorithm,500.0,0.5,0.033036
854,Random Propagation Algorithm,500.0,0.4,0.082938
855,Random Propagation Algorithm,500.0,0.6,0.021328
